In [1]:
%env WANDB_API_KEY=ca052f00ea6de6ee4031c1b94b78629919d4191c

# General imports
import os
import json
import collections

# Data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import scipy.sparse as sp

import torch
from torch_geometric.loader import DataLoader
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
from scipy.sparse import coo_matrix
import copy

from torch_geometric.utils import to_networkx

import networkx as nx
from networkx.algorithms import community

from tqdm.auto import trange

from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from collections import defaultdict

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
import torch.nn.functional as F
from scipy import sparse
import wandb
wandb.login()



env: WANDB_API_KEY=ca052f00ea6de6ee4031c1b94b78629919d4191c


/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_geometric/typing.py:63: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: dlopen(/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_scatter/_scatter_cpu.so, 0x0006): Symbol not found: __ZN3c106SymInt19promote_to_negativeEv
  Referenced from: <6B2EC669-0D00-3C24-BAA9-93003D8299BC> /Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_scatter/_scatter_cpu.so (built for macOS 13.5.1 which is newer than running OS)
  Expected in:     <1265BA86-DC1B-3079-8E88-530FC246767C> /Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch/lib/libc10.dylib
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_geometric/typing.py:74: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: dlopen

OSError: dlopen(/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_cluster/_grid_cpu.so, 0x0006): Symbol not found: __ZN3c106SymInt19promote_to_negativeEv
  Referenced from: <7C703E6E-3D2A-3F90-BEBA-7406D017F238> /Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_cluster/_grid_cpu.so (built for macOS 13.5.1 which is newer than running OS)
  Expected in:     <1265BA86-DC1B-3079-8E88-530FC246767C> /Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch/lib/libc10.dylib

In [2]:
import pickle

with open('incongruent_graphs.pickle', 'rb') as output:
    graphs = pickle.load(output)


with open('incongruent_labels.pickle', 'rb') as output:
      label_array = pickle.load(output)


with open('incongruent_groups.pickle', 'rb') as output:
      group_array = pickle.load(output)   


In [3]:
unique, counts = np.unique(label_array, return_counts=True)
print(np.asarray((unique, counts)).T)

[[  0 341]
 [  1 244]]


In [4]:
use_wandb = True #@param {type:"boolean"}
wandb_project = "gnn_stroop" #@param {type:"string"}
wandb_run_name = "dataset-incongruent" #@param {type:"string"}

if use_wandb:
    wandb.init(project=wandb_project, name=wandb_run_name)

In [22]:
# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

In [5]:
from torch_geometric.data import InMemoryDataset, Data
import torch
import numpy as np

def create_pytorch_data(data_list, labels):
        data_list_pyg = []  # List to store PyG Data objects

        for graph_data, label in zip(data_list, labels):

            # Convert NumPy arrays to PyTorch tensors
            coo = coo_matrix(graph_data)

            # Extract non-zero entries as edge tuples
            edge_index = torch.tensor([np.array(coo.row), np.array(coo.col)], dtype=torch.long)

            edge_weight = torch.from_numpy(coo.data).to(torch.float32)
            y = torch.tensor(label, dtype=torch.long)  # Class for each graph

            # Create a PyG Data object for each graph in your dataset (without node features)
            data = Data(edge_index=edge_index, edge_weight=edge_weight, y=y)
            data_list_pyg.append(data)

        # Concatenate the PyG Data objects into a single Data object
        # data = collate(data_list_pyg)
        return data_list_pyg


In [6]:
# Create an instance of your custom dataset
custom_dataset = create_pytorch_data(data_list=graphs, labels = label_array)

# Access the processed PyG Data objects
data = custom_dataset[0]  # Get the first graph in the dataset
print(data)
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

num_nodes = data.num_nodes


Data(edge_index=[2, 13], y=0, edge_weight=[13])
Number of nodes: 4
Number of edges: 13
Average node degree: 3.25
Has self-loops: True
Is undirected: True


/var/folders/x4/6xjb_tfs6vdg6llm0_l4vjz00000gn/T/ipykernel_69041/122919934.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  edge_index = torch.tensor([np.array(coo.row), np.array(coo.col)], dtype=torch.long)
/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'y', 'edge_index', 'edge_weight'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [ ]:
data_details = {
    "num_nodes": {data.num_nodes},
    "num_edges": {data.num_edges},
    "Is undirected": {data.is_undirected()}
}

if use_wandb:
    # Log all the details about the data to W&B.
    wandb.log(data_details) #🪄🐝
else:
    print(json.dumps(data_details, sort_keys=True, indent=4))

In [104]:
# 16 graphs * 17 partipants 
# 14 training and 3 testing

#GCN example 3   

training_graphs= 12 * 39

train_dataset = custom_dataset[:training_graphs]
test_dataset = custom_dataset[training_graphs:]

torch.manual_seed(12345)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 468
Number of test graphs: 117


In [105]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 15], y=[1], edge_weight=[15], batch=[4], ptr=[2])

Step 2:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 13], y=[1], edge_weight=[13], batch=[4], ptr=[2])

Step 3:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 14], y=[1], edge_weight=[14], batch=[4], ptr=[2])

Step 4:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 14], y=[1], edge_weight=[14], batch=[4], ptr=[2])

Step 5:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 13], y=[1], edge_weight=[13], batch=[4], ptr=[2])

Step 6:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 12], y=[1], edge_weight=[12], batch=[4], ptr=[2])

Step 7:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 12], y=[1], edge_weight=[12], batch=[4], ptr=[2])

Step 8:
Number of graphs in the current batch: 1
DataBatch(edge_index=[2, 12], y=[1], edge_weight=[12], batch=[4], ptr=[2])



In [32]:
import torch.nn as nn
import torch

class GCNModel(nn.Module):
    def __init__(self, num_nodes, hidden_size=32):
        super(GCNModel, self).__init__()
        self.node_embedding = nn.Embedding(num_nodes, hidden_size)
        # Initialize the embeddings with small random values
        nn.init.normal_(self.node_embedding.weight, std=0.1)
        self.conv1 = ...  # Your graph convolutional layer here

    def forward(self, edge_index, edge_attr, batch):
        x = self.node_embedding.weight
        x = self.conv1(x, edge_index, edge_attr)
        ...
        return x

In [112]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GraphConv


class GCN(torch.nn.Module):
    def __init__(self,  num_nodes, hidden_channels, num_classes):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.node_embedding = nn.Embedding(num_nodes, hidden_channels)
        # Initialize the embeddings with small random values
        nn.init.normal_(self.node_embedding.weight, std=0.1)
        self.conv1 = GraphConv(hidden_channels, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, edge_index, edge_weight, batch):
         # 1. Obtain node embeddings 
        x = self.node_embedding.weight
       
        x = self.conv1(x=x, edge_index=edge_index, edge_weight=edge_weight)
        x = x.relu()
        x = self.conv2(x=x, edge_index=edge_index, edge_weight=edge_weight)
        x = x.relu()
        x = self.conv3(x=x, edge_index=edge_index, edge_weight=edge_weight)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(num_nodes = num_nodes, hidden_channels=64, num_classes = 2)
print(model)

GCN(
  (node_embedding): Embedding(4, 64)
  (conv1): GraphConv(64, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [113]:
data
model = GCN(num_nodes = num_nodes, hidden_channels=16, num_classes = 2)
out = model(data.edge_index, data.edge_weight,  data.batch)  # Perform a single forward pass
print(out)

tensor([[ 0.0619, -0.4636]], grad_fn=<AddmmBackward0>)


In [76]:
data.edge_index

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7],
        [1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2, 5, 6, 7, 4, 6, 7, 4, 5, 7, 4, 5, 6]])

In [67]:
custom_dataset[0].edge_index

tensor([[0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
        [0, 1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2]])

In [115]:
model = GCN(num_nodes = num_nodes, hidden_channels=32, num_classes = 2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.edge_index, data.edge_weight,  data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.edge_index, data.edge_weight,  data.batch)
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 002, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 003, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 004, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 005, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 006, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 007, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 008, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 009, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 010, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 011, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 012, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 013, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 014, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 015, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 016, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 017, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 018, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 019, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 020, Train Acc: 0.6004, Test Acc: 0.5128
Epoch: 021, Train Acc: 0.6004, Test Acc:

In [28]:
from torch_geometric.utils import to_networkx
import networkx as nx


G = to_networkx(data, to_undirected=True)
# visualize_graph(G, color=data.attribute)
nx.draw(G)

/Users/erangad/opt/miniconda3/envs/env/lib/python3.10/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_attr', 'edge_index', 'y'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

In [46]:
test_graph = graphs[0]
# test_graph

# r_mat_np_coo = sparse.coo_matrix(test_graph)

coo = coo_matrix(test_graph)

# Extract non-zero entries as edge tuples
edge_index = torch.tensor([np.array(coo.row), np.array(coo.col)], dtype=torch.long)
print(edge_index)
print(coo.data)

edge_attr = torch.from_numpy(coo.data).to(torch.float32)
edge_attr

tensor([[0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3],
        [0, 1, 2, 3, 0, 2, 3, 0, 1, 3, 0, 1, 2]])
[-1.11022302e-16  4.54911676e-02  2.51465435e-01  5.51392897e-01
  4.54911676e-02  2.21198307e-01  4.72674532e-01  2.51465435e-01
  2.21198307e-01  2.82254092e-01  5.51392897e-01  4.72674532e-01
  2.82254092e-01]


tensor([-1.1102e-16,  4.5491e-02,  2.5147e-01,  5.5139e-01,  4.5491e-02,
         2.2120e-01,  4.7267e-01,  2.5147e-01,  2.2120e-01,  2.8225e-01,
         5.5139e-01,  4.7267e-01,  2.8225e-01])

In [ ]:
# Example NumPy dataset (Replace this with your actual NumPy dataset)
numpy_data = [
    {'edge_index': np.array([[0, 1, 1, 2, 2, 3], [1, 0, 2, 1, 3, 2]]),
     'edge_attr': np.random.rand(6, 2), 'class': 0},  # Class 0 for the first graph
    {'edge_index': np.array([[0, 1, 1, 2, 2, 3, 3, 4], [1, 0, 2, 1, 3, 2, 4, 3]]),
     'edge_attr': np.random.rand(8, 2), 'class': 1},  # Class 1 for the second graph
    # Add more dictionaries representing graphs in your dataset
]

# Create an instance of your custom dataset
custom_dataset = CustomNumpyGraphDataset(data_list=numpy_data)

# Access the processed PyG Data objects
data = custom_dataset[0]  # Get the first graph in the dataset
print(data)

In [ ]:
# load edges between users and movies
def load_edge_csv(df, 
                  src_index_col, 
                  dst_index_col,  
                  link_index_col, 
                  rating_threshold=3.5):
    """Loads csv containing edges between users and items

    Args:
        path (str): path to csv file
        src_index_col (str): column name of users
        src_mapping (dict): mapping between row number and user id
        dst_index_col (str): column name of items
        dst_mapping (dict): mapping between row number and item id
        link_index_col (str): column name of user item interaction
        rating_threshold (int, optional): Threshold to determine positivity of edge. Defaults to 4.

    Returns:
        torch.Tensor: 2 by N matrix containing the node ids of N user-item edges
    """
    
    edge_index = None
    src = [user_id for user_id in  df['userId']]
    
    num_users = len(df['userId'].unique())

    dst = [(movie_id) for movie_id in df['movieId']]
    
    link_vals = df[link_index_col].values

    edge_attr = torch.from_numpy(df[link_index_col].values).view(-1, 1).to(torch.long) >= rating_threshold

    edge_values = []

    edge_index = [[], []]
    for i in range(edge_attr.shape[0]):
        if edge_attr[i]:
            edge_index[0].append(src[i])
            edge_index[1].append(dst[i])
            edge_values.append(link_vals[i])

    # edge_values is the label we will use for compute training loss                
    return edge_index, edge_values

In [ ]:
def convert_r_mat_edge_index_to_adj_mat_edge_index(input_edge_index, input_edge_values):
    R = torch.zeros((num_users, num_movies))
    for i in range(len(input_edge_index[0])):
        row_idx = input_edge_index[0][i]
        col_idx = input_edge_index[1][i]
        R[row_idx][col_idx] = input_edge_values[i] # assign actual edge value to Interaction Matrix

    R_transpose = torch.transpose(R, 0, 1)
    
    # create adj_matrix
    adj_mat = torch.zeros((num_users + num_movies , num_users + num_movies))
    adj_mat[: num_users, num_users :] = R.clone()
    adj_mat[num_users :, : num_users] = R_transpose.clone()
    
    adj_mat_coo = adj_mat.to_sparse_coo()
    adj_mat_coo_indices = adj_mat_coo.indices()
    adj_mat_coo_values = adj_mat_coo.values()
    return adj_mat_coo_indices, adj_mat_coo_values

In [ ]:
def convert_adj_mat_edge_index_to_r_mat_edge_index(input_edge_index, input_edge_values):
    
    sparse_input_edge_index = SparseTensor(row=input_edge_index[0], 
                                           col=input_edge_index[1], 
                                           value = input_edge_values,
                                           sparse_sizes=((num_users + num_movies), num_users + num_movies))
    
    adj_mat = sparse_input_edge_index.to_dense()
    interact_mat = adj_mat[: num_users, num_users :]
    
    r_mat_edge_index = interact_mat.to_sparse_coo().indices()
    r_mat_edge_values = interact_mat.to_sparse_coo().values()
    
    return r_mat_edge_index, r_mat_edge_values

In [ ]:
edge_index, edge_values = load_edge_csv(
    rating_df,
    src_index_col='userId',
    dst_index_col='movieId',
    link_index_col='rating',
    rating_threshold=1 # need to use threshold=1 so the model can learn based on RMSE
)

In [ ]:
# Convert to tensor
# We use LongTensor here because the .propagate() method in the model needs either LongTensor or SparseTensor
edge_index = torch.LongTensor(edge_index) 
edge_values = torch.tensor(edge_values)

print(edge_index)
print(edge_index.size())

print(edge_values)
print(edge_values.size())

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

In [ ]:
# gscv.best_score_
from sklearn.model_selection import LeaveOneGroupOut

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold,GridSearchCV,cross_val_score,cross_validate 
logo = LeaveOneGroupOut()

from torch.utils.data import DataLoader

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

NameError: name 'dataset' is not defined

In [ ]:
accuracy=[]
for train_index, val_index in logo.split(data_array, label_array, groups=group_array):
    train_features,train_labels=data_array[train_index],label_array[train_index]
    val_features,val_labels=data_array[val_index],label_array[val_index]
    scaler=StandardScaler()
    train_features = scaler.fit_transform(train_features.reshape(-1, train_features.shape[-1])).reshape(train_features.shape)
    val_features = scaler.transform(val_features.reshape(-1, val_features.shape[-1])).reshape(val_features.shape)

    data_train = torch.from_numpy(train_features).float()
    label_train = torch.from_numpy(train_labels).float()

    data_val = torch.from_numpy(val_features).float()
    label_val = torch.from_numpy(val_labels).float()

    print('Training:', data_train.size(), label_train.size())
    print('Val:', data_val.size(), label_val.size())

    train( data_train,label_train,
                    data_val, label_val)